In [ ]:
import pandas 
import re 
from telethon import TelegramClient

In [ ]:
async def parser(list_channel, tematic):

  api_id =   # задаем API
  api_hash = ''  #задаем HASH
  post_count = 2000
  data = []

  async with TelegramClient('session_name', api_id, api_hash) as client:

        for channel_name in list_channel:

          async for message in client.iter_messages(channel_name, reverse=False, limit=post_count):

            if message.message:
              text=message.message
              data.append({'Text': text})

        client.disconnect()

  df = pandas.DataFrame(data)
  df['Thematics'] = tematic
  return df

In [8]:
def low_reg(x): #приводит текст к нижнему регистру
    x = x.lower()
    return x

def remove_emails(x): #удаляет смайлики
    return re.sub(r'([a-z0-9+._-]+@[a-z0-9+._-]+\.[a-z0-9+_-]+)',"", x)

def remove_urls(x): #удаляет ссылки
    return re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , x)

def remove_rt(x):
    return re.sub(r'\brt\b', '', x).strip()

def remove_special_chars(x):
    x = re.sub(r'[^\w ]+', "", x)
    x = ' '.join(x.split())
    return x

In [5]:
def redactor_txt(df):

  df['Text'] = df['Text'].apply(lambda x: low_reg(x))
  df['Text'] = df['Text'].apply(lambda x: remove_emails(x))
  df['Text'] = df['Text'].apply(lambda x: remove_urls(x))
  df['Text'] = df['Text'].apply(lambda x: remove_rt(x))
  df['Text'] = df['Text'].apply(lambda x: remove_special_chars(x))

  df = df.dropna(axis = 0, how ='any')

  return df

In [7]:
def load_channel_data(filepath):

    try:
        return pandas.read_csv(filepath)
    except FileNotFoundError:
        print(f"Ошибка: Файл {filepath} не найден.")
        return None

def list_for_parser(df_channels, tematic_name):

    channels = df_channels.loc[df_channels['Thematics'] == tematic_name, 'Channels']
    if channels.empty:
        return []

    channel_list = channels.tolist()
    if not channel_list:
        return []

    try:
        return channel_list[0].split()
    except AttributeError:
        print(f"Ошибка: Некорректные данные в столбце 'Channels' для тематики '{tematic_name}'.")
        return []
    except Exception as e:
        print(f"Непредвиденная ошибка при обработке каналов: {e}")
        return []

In [8]:
def get_channels(id, df_name_channels):
  tematics = {
      0: 'Новости и СМИ',
      1: 'Блоги',
      2: 'Технологии',
      3: 'Юмор и развлечения',
      4: 'Политика',
      5: 'Экономика и финансы',
      6: 'Криптовалюты',
      7: 'Наука и образование',
      8: 'Музыка',
      9: 'Лингвистика',
      10: 'Бизнес и стартапы',
      11: 'Психология',
      12: 'Маркетинг и реклама',
      13: 'Карьера',
      14: 'Фильмы и сериалы',
      15: 'Литература',
      16: 'Спорт',
      17: 'Здоровье',
      18: 'Путешествия и эмиграция',
      19: 'Искусство и фото',
      20: 'Мода и красота',
      21: 'Медицина',
      22: 'Игры и приложения',
      23: 'Еда и напитки',
      24: 'Продажи',
      25: 'Автомобили',
      26: 'Недвижимость',
      27: 'Животные',
      28: 'Религия',
      29: 'Мебель и дизайн интерьеров',
      30: 'Мистика и эзотерика'
  }



  return list_for_parser(df_name_channels, tematics[id]), tematics[id]

In [11]:
async def main():

  df = pandas.DataFrame()
  df_result = pandas.DataFrame()
  list_channel = []

  filepath_list_chanell = 'list_Channels.csv' # кастомный датасет со списком каналов и их тематиками
  df_name_channels = load_channel_data(filepath_list_chanell)

  for i in range(30):

    list_channel, tematic = get_channels(i, df_name_channels)

    if not list_channel:
      continue

    df = await parser(list_channel, tematic)
    df = redactor_txt(df)
    df_result = pandas.concat([df, df_result])

  df_result.to_csv('train_dataset_classifier.csv')

In [ ]:
await main()